## Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Source #Partner

### Links: 
___

In [17]:
# creating a table from s3

query_text = """--sql
    CREATE TABLE db1.no_video_on_stream_mobile_st_asgard
    (
        `report_date` Date,
        `camera_uuid` String,
        `count` Int32
    )
    ENGINE = S3('https://storage.yandexcloud.net/dwh-asgard/no_video_on_stream_mobile_st_asgard/year=*/month=*/*.csv','CSVWithNames')
    """

ch.query_run(query_text)

DatabaseError: HTTPDriver for https://rc1a-eflrt968scbm232q.mdb.yandexcloud.net:8443 received ClickHouse error code 57
 Code: 57. DB::Exception: Table db1.no_video_on_stream_mobile_st_asgard already exists. (TABLE_ALREADY_EXISTS) (version 25.3.6.56 (official build))


In [20]:
# creating a table for materialized view

query_text = """--sql
    CREATE TABLE db1.no_video_on_stream_mobile_st_asgard_ch
    (
        `report_date` Date,
        `camera_uuid` String,
        `count` Int32
    )
    ENGINE = MergeTree()
    ORDER BY camera_uuid
    """

ch.query_run(query_text)

""


In [21]:
# creating a materialized view

query_text = """--sql
    CREATE MATERIALIZED VIEW db1.no_video_on_stream_mobile_st_asgard_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR RANDOMIZE FOR 1 HOUR TO db1.no_video_on_stream_mobile_st_asgard_ch AS
    SELECT
        *
    FROM db1.no_video_on_stream_mobile_st_asgard
    """

ch.query_run(query_text)

""


___
## Tools
___
### query

In [25]:
query_text = """--sql
    SELECT
        *
    FROM db1.no_video_on_stream_mobile_st_asgard_ch
    ORDER BY report_date desc
    LIMIT 2
    """

ch.query_run(query_text)

,report_date,camera_uuid,count
0,2025-09-14,01b1689a-a4b8-4070-b2cd-962da6406e60,0
1,2025-09-14,01ac5223-d082-438c-b798-2ec03f2ef322,0


### Drop ch

In [19]:
query_text = """
    DROP TABLE db1.no_video_on_stream_mobile_st_asgard_ch
    """
ch.query_run(query_text)

,read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
0,0,0,0,0,0,0,0,1746600,cdae6480-f356-4e22-ac15-e31a15ce7bb8


### Drop mv

In [26]:
query_text = """
    DROP TABLE db1.no_video_on_stream_mobile_st_asgard_mv
    """
ch.query_run(query_text)

,read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
0,0,0,0,0,0,0,0,1252849,ef5fbd13-073a-4452-9124-e1ee47fe65bf
